In [3]:
import pandas as pd
from collections import Counter

In [4]:
filepath="/home/mitsuki/out/altorf/proteome/result/Arthrobacter_sp_FB24/Arthro_009-LTQFT3_04Oct05_Andro_0705-06_ocelot166.tsv"
df=pd.read_csv(filepath,sep='\t')
fdf=df[df["PepQValue"]<=0.05]
fdf.head()

,#SpecFile,SpecID,ScanNum,FragMethod,Precursor,IsotopeError,PrecursorError(ppm),Charge,Peptide,Protein,DeNovoScore,MSGFScore,SpecEValue,EValue,QValue,PepQValue
0,Arthro_009-LTQFT3_04Oct05_Andro_0705-06.mzML,controllerType=0 controllerNumber=1 scan=9296,9296,CID,1129.2174,2,-0.503531,3,VLSGEEPFDEQTMIAGHHVEQEVAAPAPAPAR,"REF|WP_011691390.1(pre=R,post=A)",225,205,7.497198e-31,4.044793e-24,0.0,0.0
1,Arthro_009-LTQFT3_04Oct05_Andro_0705-06.mzML,controllerType=0 controllerNumber=1 scan=9630,9630,CID,1466.9968,1,-1.483788,3,ANLFTGPSAFSDFSYDTLGGDGAPEFHAIPLDDYDLGSDFR,"REF|WP_011692815.1(pre=K,post=-)",236,175,2.482416e-30,1.340034e-23,0.0,0.0
2,Arthro_009-LTQFT3_04Oct05_Andro_0705-06.mzML,controllerType=0 controllerNumber=1 scan=9172,9172,CID,1395.0171,1,10.080626,3,NDPVTALTYAQQAHALAAQAAQLAQGDVDSFGYTNQGYGR,"REF|WP_043430840.1(pre=R,post=G)",297,215,1.747156e-29,9.430822e-23,0.0,0.0
3,Arthro_009-LTQFT3_04Oct05_Andro_0705-06.mzML,controllerType=0 controllerNumber=1 scan=3840,3840,CID,1049.4988,1,0.136410,3,AAQSAAGPASAGTDASVPNSGPSGEPGGPQVPEAR,"REF|WP_011693732.1(pre=R,post=Q)",238,210,6.286698e-29,3.392423e-22,0.0,0.0
4,Arthro_009-LTQFT3_04Oct05_Andro_0705-06.mzML,controllerType=0 controllerNumber=1 scan=9136,9136,CID,1128.8877,1,3.696283,3,VLSGEEPFDEQTMIAGHHVEQEVAAPAPAPAR,"REF|WP_011691390.1(pre=R,post=A)",227,194,1.114230e-28,6.011354e-22,0.0,0.0


In [5]:
def judge_origin(protein):
    def parse_identifier(identifier):
        assert len(identifier)==3
        if identifier.upper()=="REF" or identifier[:2]=="gb" or identifier[:2]=="gi":#not allways REF, when _msgfplus
            return "REF"
        elif identifier=="ORF":
            return "ORF"
        elif identifier=="OC1":
            return "OC1"
        elif identifier=="OC5":
            return "OC5"
        else:#not allways CON, when _msgfplus
            return "CON"
    
    if protein[:3]=="XXX":#is decoy
        return 'D'+parse_identifier(protein[4:7])
    else:
        return parse_identifier(protein[:3])

In [6]:
origin_lst=[judge_origin(i) for i in fdf["Protein"]]
counter=Counter(origin_lst)

In [7]:
counter

Counter({'CON': 37,
         'DOC1': 33,
         'DOC5': 28,
         'DREF': 62,
         'OC1': 27,
         'OC5': 25,
         'REF': 5769})

In [ ]:
origin_lst=[judge_origin(i) for i in fmsgf_df["Protein"]]
counter=Counter(origin_lst)
print(counter)

In [ ]:
origin_lst=[judge_origin(i) for i in f_df["Protein"]]
counter=Counter(origin_lst)
print(counter)

In [ ]:
df.head()

In [ ]:
filename="/home/mitsuki/altorf/proteome/createcatalog/pnnl_downloads.csv"
catalogDf=pd.read_csv(filename)

dct_lst=[]
for key, row in catalogDf.iterrows():
    org=row["Organism"]
    data=row["Dataset"]
    
    fastaType_lst=["msgfplus","annotated"]
    dir_lst=["/home/mitsuki/data/pnnl/massive.ucsd.edu/result/"+org+"/",
             "/home/mitsuki/out/altorf/proteome/result/"+org+"/"]
    filepath_lst=(dir_lst[0]+data+"_msgfplus.tsv",
                  dir_lst[1]+data+"_annotated.tsv")
    
    counter_lst=[]
    for filepath in filepath_lst:
        df=pd.read_csv(filepath,sep='\t')
        fdf=df[df["PepQValue"]<=0.05]#filter by peptide level FDR 0.05
        origin_lst=[judge_origin(i) for i in fdf["Protein"]]
        counter=Counter(origin_lst)
        counter_lst.append(counter)
        
    for i,fastaType in enumerate(fastaType_lst):
        dct={}
        dct["Organism"]=org
        dct["Dataset"]=data
        dct["fasta_type"]=fastaType
        dct["REF"]=counter_lst[i]["REF"]
        dct["CON"]=counter_lst[i]["CON"]
        dct["XXX"]=counter_lst[i]["XXX"]
        dct_lst.append(dct)
        
    print("DONE "+org+'/'+data)

In [ ]:
counter_lst

In [ ]:
dct_lst=[]
for i,fastaType in enumerate(fastaType_lst):
    dct={}
    dct["Organism"]=org
    dct["Dataset"]=data
    dct["fasta_type"]=fastaType
    dct["REF"]=counter_lst[i]["REF"]
    dct["CON"]=counter_lst[i]["CON"]
    dct["XXX"]=counter_lst[i]["XXX"]
    dct_lst.append(dct)

In [ ]:
dct_lst

In [ ]:
out_df=pd.DataFrame(dct_lst)
out_df=out_df[["Organism","Dataset","fasta_type","REF","CON","XXX"]]
out_df.tail(10)